In [ ]:
from evoframe.reward_builders import RewardBuilderGame
from evoframe.population_update_builders import PopulationUpdateBuilderStatic
from evoframe.selector_function import SelectorFunctionFactory
from evoframe import EvolutionBuilder
from evoframe.models import FeedForwardNetwork
from evoframe.models import ActivationFunctions
from evoframe.games import Game
from evoframe import get_agent_wrapper_func

import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from evoframe.experiment_results import *
from evoframe.utility import clean_experiment_directory

%matplotlib inline

In [ ]:
from evoframe.reward_builders.reward_builder_game import TournamentMode
from evoframe.games import Tris

In [ ]:
# Game
game_creation_func = lambda context: Tris()

# Model
layer_sizes = [9, 6, 3, 1]
get_model_func = lambda: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_sigmoid(), ActivationFunctions.get_sigmoid())

# Game-Model interface
def predict_func(model, game):
    available_actions = game.get_available_actions()
    vs = []
    for a in available_actions:
        next_state = game.get_next_state(a)
        vs.append(model.predict(next_state.flatten())[0])
    i_max = np.array(vs).argmax()
    best_action = available_actions[i_max]
    return best_action
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
# Since Tris is a two-players game, we can compute the reward with a tournament against the current generation
keep_only = 30
reward_func, get_context_func = RewardBuilderGame() \
.with_game_creation_func(game_creation_func) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_competitive_tournament(TournamentMode.VS_BESTS_RANDOM) \
.with_keep_only(keep_only) \
.get()

# Update population function
get_new_pop_func = PopulationUpdateBuilderStatic() \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.05, 0.1, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.05, 0.2, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.05, 0.3, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.1, 0.1, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.1, 0.2, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.1, 0.3, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.2, 0.1, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.2, 0.2, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.2, 0.3, 1) \
.add_selector_func(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.get() # learning rate, sigma, keep perc 

# Evolution function
evolution_func = EvolutionBuilder() \
    .with_get_model_func(get_model_func) \
    .with_reward_func(reward_func) \
    .with_get_new_pop_func(get_new_pop_func) \
    .with_get_context_func(get_context_func) \
    .get()

pop_size = 100
num_epochs = 10
experiment_name = "tris_gradient_and_mutation"
clean_experiment_directory(experiment_name)
evolution_func(experiment_name, pop_size, num_epochs, num_threads=8)

In [ ]:
pop_size = 100
num_epochs = 10
experiment_name = "tris_gradient_and_mutation"
def predict_func(model, game):
    available_actions = game.get_available_actions()
    vs = []
    for a in available_actions:
        next_state = game.get_next_state(a)
        vs.append(model.predict(next_state.flatten())[0])
    i_max = np.array(vs).argmax()
    best_action = available_actions[i_max]
    return best_action
agent_wrapper_func = get_agent_wrapper_func(predict_func)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
# import plotly.express as px

def show_predictions(model, inp):
    game = Tris()
    game.board = inp
    out = np.array(model.predict(game))
    for r,row in enumerate(game.board):
        for c,cell in enumerate(row):
            if cell != 0:
                out[r*3 + c] = -100000
    out = out.reshape((3,3))
    shape = out.shape
    data = [(row+1, col+1, out[row][col]) for row in range(shape[0]) for col in range(shape[1])]
    columns = ["neuron_input", "neuron_output", "value"]
    df = pd.DataFrame(data=data, columns=columns)
    return px.density_heatmap(df, x="neuron_output", y="neuron_input", z="value",
                                         histfunc="sum", color_continuous_scale="RdYlGn", range_color=(-2,2),
                                         nbinsx=shape[1], nbinsy=shape[0],
                                         range_x=(0.5, shape[1]+0.5), range_y=(0.5, shape[0]+0.5))

In [ ]:
def get_random_input_func():
    return np.array([[np.random.choice([-1,0,1]) for i in range(3)] for j in range(3)])

In [ ]:
model = agent_wrapper_func(pickle_load_best_model_of_epoch(experiment_name, num_epochs, pop_size))
inp = get_random_input_func()
for i in range(3):
    for j in range(3):
        print(inp[2-i][j], end=" ")
    print("")
show_predictions(model, inp)

In [ ]:
plot_behavioural_differences(experiment_name, get_random_input_func, mode="first_best")

In [ ]:
plot_behavioural_differences(experiment_name, get_random_input_func, mode="last_best")

In [ ]:
plot_behavioural_variances_to_input(experiment_name, get_random_input_func)

In [ ]:
figs = plot_params_statistics(experiment_name)

In [ ]:
figs[0]

In [ ]:
figs[1]

In [ ]:
figs[2]

In [ ]:
figs[3]

In [ ]:
plot_operators_statistics_means(experiment_name)

In [ ]:
plot_operators_statistics_max(experiment_name)

In [ ]:
plot_tournament(experiment_name, agent_wrapper_func, Tris, 2)[0]

In [ ]:
class AgentHuman:
    def __init__(self):
        pass
    
    def predict(self, game):
        move = int(input("Select move: "))
        prediction = [1 if i == move else 0 for i in range(9)]
        return prediction

In [ ]:
game = Tris()
game.play(AgentHuman(), agent_wrapper_func(pickle_load_best_model_of_epoch(experiment_name, num_epochs, pop_size)), interactive=True)